In [37]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings; warnings.filterwarnings('ignore')

# data 불러오기
def load_data():
    global df_main, df2
    
    df1 = pd.read_csv('./data/df.csv')
    df2 = pd.read_csv("./data/weather.csv")
    
    # 필요한 부분만 추출
    df_main = df1[ ['ID','이름', '대분류','소분류', '평점', '투표횟수','실내/실외', '태깅'] ]

    return df_main, df2


In [38]:
def countVec():
    # CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
    count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
    facility_mat = count_vect.fit_transform( df_main['태깅'] )
    facility_sim = cosine_similarity(facility_mat, facility_mat)

    # 유사도가 높은 순으로 정리된 facility_sim 객체의 비교 행 위치 인덱스 값
    # 값이 높은 순으로 정렬된 비교 대상 행의 유사도 값이 아니라
    # 비교 대상 행의 위치 인덱스임에 주의
    facility_sim_sorted_ind = facility_sim.argsort()[:, ::-1]
    
    return facility_sim_sorted_ind

In [39]:
# 기존 평점을 가중 평점으로 변경하는 함수
def weighted_vote_average(record):
  C = df_main['평점'].mean()
  m = df_main['투표횟수'].quantile(0.6)
  
  v = record['투표횟수']
  R = record['평점']
  # (예정)날씨 관련 수식을 추가 => 실내외 구분시 활용
  return ( ( (v/(v+m)) * R ) + ( (m/(v+m)) * C ) ) *2


In [40]:
# 새롭게 정의된 평점 기준에 따라 기존 find_sim_experience 함수를 변경
def find_sim_experience(df_main, sorted_ind, title_name, top_n=10, end_idx=10):
  df_main['추천점수'] = df_main.apply(weighted_vote_average, axis=1)
  title_exp = df_main[df_main['이름'] == title_name]
  title_index = title_exp.index.values

  sim_index = sorted_ind[title_index, :top_n*3]
  sim_index = sim_index.reshape(-1)

  sim_index = sim_index[sim_index != title_index]

  return df_main.iloc[sim_index].sort_values(
    '추천점수', ascending=False)[:end_idx]

# 우천시 추천리스트 10개 출력
def find_sim_experience_rainy(df_main, sorted_ind, title_name, top_n=10):
  df_main['추천점수'] = df_main.apply(weighted_vote_average, axis=1)
  title_exp = df_main[df_main['이름'] == title_name]
  title_index = title_exp.index.values

  sim_index = sorted_ind[title_index, :top_n*3]
  sim_index = sim_index.reshape(-1)

  sim_index = sim_index[sim_index != title_index]

  return df_main.iloc[sim_index].sort_values(
    '추천점수', ascending=False)[:]


In [41]:
def input_title(input_name):
    facility_sim_sorted_ind = countVec()
    
    sim_exp = find_sim_experience(df_main, facility_sim_sorted_ind, input_name)
    sim_exp_rainy = find_sim_experience_rainy(df_main, facility_sim_sorted_ind, input_name)
    result_sunny = sim_exp[['이름', '대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']]
    result_rain = sim_exp_rainy[['이름', '대분류', '소분류', '실내/실외', '평점', '투표횟수', '추천점수']]

    return result_sunny, result_rain

In [42]:
# 날씨기반 실내/외 판별 함수
# temper = 온도 , humid = 습도 , rain = 1시간당 강수량

def weather_choice(temper,humid,rain):
  y = humid - ((-4.3 * temper) + 147)
  if y >= 10:
    temper = temper + (y/10)

  if rain >= 5 or temper<=10 or temper>=30:
    return '실내'
  else:
    return '실외'
    

In [43]:
def run_model(input_name, weather=0):
    # weather = 맑음 : 0 (default) / 비 : 1    
    
    _, df2 = load_data()
    result_sunny, result_rain = input_title(input_name)
    
    # 날씨로 실내/실외 필터링
    
    temper = df2.loc[weather][0:][0] # 온도
    humid = df2.loc[weather][0:][1]  # 습도
    rain = df2.loc[weather][0:][2]   # 강수량
        
        # 비 -> 실내만 추천
    if weather_choice(temper, humid, rain) == "실내":
        return result_rain[result_rain['실내/실외'] == "실내"][:10]
        
        # 맑음 -> 실내/외 모두 추천
    else :
        return result_sunny

run_model('국립광주박물관')

,이름,대분류,소분류,실내/실외,평점,투표횟수,추천점수
32,국립아시아문화전당,예술,공연장,실내,4.6,279,8.627540
33,유ㆍ스퀘어문화관,예술,공연장,실내,4.3,2160,8.532185
35,광주문화예술회관,예술,공연장,실내,4.3,1040,8.467752
2,광주시립미술관,예술,미술관,실내,4.3,815,8.436555
23,광주광역시역사민속박물관,예술,박물관,실내,4.3,298,8.243101
43,월봉서원,예술,전통문화,실내,4.5,117,8.173520
30,미디어놀이터,예술,공연장,실내,4.3,213,8.156869
40,빛고을시민문화관,예술,공연장,실내,4.3,213,8.156869
37,김대중컨벤션센터,예술,공연장,실내,4.1,2147,8.156088
19,하정웅미술관,예술,미술관,실내,4.3,182,8.114047
